In [127]:
#coding: utf-8
# citations: https://pythonmana.com/2021/03/20210329161147051K.html
# citation: https://stackoverflow.com/questions/43637211/retrieve-document-content-with-document-structure-with-python-docx

# Import Libraries
!pip install docx
from docx import Document
from docx.document import Document as _Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
import pandas as pd
import numpy as np
import re
from os.path import exists
!pip install xlrd

In [128]:
########################################################################
# USER INPUTS
########################################################################
# Replace File Path. This is where all the relevant files should be saved.
path = 'C:/Users/jstockham/OneDrive - American Institutes for Research in the Behavioral Sciences/JSmith/PYTHON_CODE_2022/table_automate_jess/general_table_automate/'

# Word Document Name (If your Word doc is tableshells.docx, report_doc = 'tableshell.docx')
report_doc = 'tableshell.docx'

# Excel Document Name (If your Excel doc is rawdata.xlsx, excel_doc = 'rawdata.xlsx')
excel_doc = 'rawdata.xlsx'

# Crosswalk Excel Document Name (If your Excel doc is crosswalk.xlsx, crosswalk_doc = 'crosswalk.xlsx')
crosswalk_doc = 'crosswalk.xlsx'  

# Name of updated Word Document - OPTIONAL.  (If your Word doc is outputfile.docx, output_doc = 'outputfile.docx')
# If not provided, will replace the contents of the inputted Word doc
output_doc = 'outputfile.docx'

In [129]:
########################################################################
# Load files
########################################################################

# 1. Load in all sheets from Excel Workbook into Pandas Dataframe
tables_data = pd.read_excel(excel_doc, sheet_name=None, header=None)
#print(tables_data["Table 1"])
#print(tables_data["Table 2"])

# 2. Load Word Document into a docx file object
document = Document(report_doc)

# 3. Load Crosswalk data into Pandas Dataframe
crosswalk_data = pd.read_excel(crosswalk_doc)
#print(crosswalk_data.head())

#4. Set output to be the same as the input Word file
if not output_doc:
    output_doc == report_doc

In [130]:
########################################################################
# Populate Table Data in Word
########################################################################
#citation: source: https://theprogrammingexpert.com/write-table-fast-python-docx/

# Loop through the paragraphs & table pairs in the Word document
''' Define script to identify table 'child' within paragraph 'parent' based on document order
    Each returned value is an instance of either Table or Paragraph. '''
def iter_block_items(parent):
    if isinstance(parent, _Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)      

'''Identifies paragraph/table object pairs and inserts the Excel data into the tables'''
def tablepopulate(document):

    # Iterate through paragraphs and table objects in document
    for block in iter_block_items(document):

        # If we encounter a paragraph with the AIR Exhibit Title Formatting
        if isinstance(block, Paragraph):

            # Check if it has Exhibit Title Formatting
            if block.style.name=="Exhibit Title":
                #print(block.text)

                # Find Exhibit Title in the Crosswalk
                exhibit_name = block.text
                excel_sheet_name = crosswalk_data.loc[crosswalk_data['word'] == exhibit_name].iloc[0, 1]

                # Continue to the next block item, which is the corresponding table
                continue

        # The table object just below it
        elif isinstance(block,Table):
            
            # loop through rows of the dataframe
            for i in range(tables_data[excel_sheet_name].shape[0]):
                # loop through columns of the dataframe
                for j in range(tables_data[excel_sheet_name].shape[1]):
                    block.cell(i,j).text = str(tables_data[excel_sheet_name].values[i,j])
                
tablepopulate(document)


#3. Save the outputted Word document
document.save(output_doc)